In [3]:
import Pkg;
Pkg.add("JuMP") 
Pkg.add("HiGHS")

   Resolving package versions...
  No Changes to `C:\Users\dieme\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\dieme\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
   Installed HiGHS_jll ─────── v1.6.0+1
   Installed PrecompileTools ─ v1.2.1
   Installed HiGHS ─────────── v1.7.0
    Updating `C:\Users\dieme\.julia\environments\v1.8\Project.toml`
⌃ [87dc4568] + HiGHS v1.7.0
    Updating `C:\Users\dieme\.julia\environments\v1.8\Manifest.toml`
⌃ [87dc4568] + HiGHS v1.7.0
⌅ [aea7be01] + PrecompileTools v1.2.1
⌅ [8fd58aa0] + HiGHS_jll v1.6.0+1
        Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated -m`
Precompiling project...
  ✓ PrecompileTools
  ✓ HiGHS_jll
  ✓ HiGHS
  3 dependencies successfully precompiled in 19 seconds. 53 already precompiled. 1 skipped during auto due to previous errors.


In [11]:
########################################################################
# ht_model_highs.jl
#
# Exemplo de coordenação hidro‐termo (versão linear simplificada) em Julia
# Usa JuMP + HiGHS como solver.
#
# -- Como usar:
# 1) Instale os pacotes:
#      ] add JuMP HiGHS
# 2) Ajuste os parâmetros (dados de usinas, demandas, etc) na seção "Dados".
# 3) Rode: julia ht_model_highs.jl
########################################################################

using JuMP
using HiGHS

# ==========================
# 1) PARÂMETROS / DADOS DE ENTRADA
# ==========================
# Número de períodos
T = 12  # por exemplo, 12 meses ou 12 semanas

# Número de usinas hidráulicas (NH) e termoelétricas (NT)
N_H = 3     # ajuste conforme a quantidade de UHs
N_T = 2     # ajuste conforme a quantidade de UTEs

# Índices
I_H = 1:N_H
I_T = 1:N_T
IT = 1:T

# ---- Dados de armazenamento e vazão para cada usina hidráulica i ----
# Volume inicial (no início de t=1), em hm³ (ou qualquer unidade consistente)
V0 = [  50.0,   # UH 1
        30.0,   # UH 2
        45.0 ]  # UH 3

# Limites de volume (hm³)
Vmin = [  20.0,   # UH 1
          10.0,   # UH 2
          15.0 ]  # UH 3

Vmax = [ 100.0,   # UH 1
         80.0,    # UH 2
         90.0 ]   # UH 3

# Vazão mínima e máxima de turbina para cada usina hidráulica (por período),
# em m³/s (ou unidade compatível com a conversão abaixo).
Qmin = [  0.0,   # UH 1
          0.0,   # UH 2
          0.0 ]  # UH 3

Qmax = [  200.0, # UH 1
         150.0,  # UH 2
         180.0 ] # UH 3

# Fator de conversão linear vazão → potência (MW por m³/s) para cada UH
# Ou seja, P^H_{i,t} = κ_i * Q_{i,t}
κ = [  0.008,   # UH 1
       0.007,   # UH 2
       0.009 ]  # UH 3

# Limite máximo de geração para cada usina hidrelétrica (em MW).
P_H_max = [ κ[i] * Qmax[i] for i in I_H ]

# ---- Dados de entrada (inflow) para cada UH i e período t ----
# Matriz Inflow[i,t] = vazão natural que chega ao reservatório
Inflow = Dict{Tuple{Int,Int}, Float64}()
for i in I_H, t in IT
    # Exemplo de inflow constante ou aleatório
    Inflow[(i,t)] =  5.0 + 2.0 * rand()  # valor em hm³ (ou unidade consistente)
end

# ---- Dados das termoelétricas ----
# Limites de geração (MW) e custo (R$/MWh) para cada termoelétrica j
P_T_min = [ 0.0,   # UTE 1
            0.0 ] # UTE 2

P_T_max = [ 200.0, # UTE 1
            150.0 ]# UTE 2

c_T = [ 150.0,    # custo R$/MWh -> UTE 1
        180.0 ]   # custo R$/MWh -> UTE 2

# ---- Demanda de energia para cada período t (MW) ----
D = [ 250.0, 260.0, 240.0, 255.0, 270.0, 265.0, 250.0, 260.0, 245.0, 255.0, 260.0, 270.0 ]

# ==========================
# 2) CRIAÇÃO DO MODELO JuMP COM HiGHS
# ==========================
model = Model(HiGHS.Optimizer)

# ==========================
# 3) VARIÁVEIS DE DECISÃO
# ==========================
@variables(model, begin
    # Para cada usina hidrelétrica i e período t:
    0 <= Q[i in I_H, t in IT] <= Qmax[i]         # vazão turbinada
    0 <= S[i in I_H, t in IT]                    # vertimento (spillage)
    V[i in I_H, t in IT]                         # volume armazenado (hm³)
    0 <= P_H[i in I_H, t in IT] <= P_H_max[i]     # potência hidráulica (MW)
    # Para cada termoelétrica j e período t:
    P_T[j in I_T, t in IT] >= 0                   # potência térmica (MW)
end)

# Restrição de limites em P_T (termoelétricas)
@constraints(model, begin
    [j in I_T, t in IT],  P_T_min[j] <= P_T[j,t] <= P_T_max[j]
end)

# ==========================
# 4) RESTRIÇÕES DO PROBLEMA
# ==========================
# 4.1) Balanço hídrico para cada usina hidráulica i e período t
for i in I_H
    @constraint(model, V[i,1] == V0[i] + Inflow[(i,1)] - Q[i,1] - S[i,1])
    for t in 2:T
        @constraint(model, V[i,t] == V[i,t-1] + Inflow[(i,t)] - Q[i,t] - S[i,t])
    end
end

# 4.2) Conversão vazão → potência hidráulica (linear simplificada)
@constraints(model, begin
    [i in I_H, t in IT],  P_H[i,t] == κ[i] * Q[i,t]
end)

# 4.3) Limites de volume em cada reservatório
@constraints(model, begin
    [i in I_H, t in IT],  Vmin[i] <= V[i,t] <= Vmax[i]
end)

# 4.4) Atender a demanda de cada período t
@constraint(model, [t in IT], sum(P_H[i,t] for i in I_H) + sum(P_T[j,t] for j in I_T) == D[t])

# ==========================
# 5) FUNÇÃO OBJETIVO
# ==========================
@objective(model, Min, sum( c_T[j] * P_T[j,t] for j in I_T, t in IT ) )

# ==========================
# 6) OTIMIZAÇÃO
# ==========================
optimize!(model)

# ==========================
# 7) EXTRAÇÃO DE RESULTADOS
# ==========================
if termination_status(model) == MOI.OPTIMAL
    println("Status: Ótimo encontrado")
    println("Custo total (Reais): ", objective_value(model))
    println()
    # Exemplo: mostrar geração de cada termoelétrica no primeiro período
    for j in I_T
        for t in IT
            println("P_T($(j),$(t)) = ", value(P_T[j,t]))
        end
    end
    println()
    for j in I_H
        for t in IT
            println("P_H($(j),$(t)) = ", value(P_H[j,t]))
        end
    end
    println()
    # Exemplo: mostrar armazenamento ao final do período T para cada UH
    for i in I_H
        for t in IT
            println("V($(i), $t) = ", value(V[i,t]))
        end
    end
else
    println("Otimização não atingiu solução ótima. Status = ",
            termination_status(model))
end


Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
48 rows, 129 cols, 198 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-144); columns 0(-168); elements 0(-333) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  4.8197041430e+05
HiGHS run time      :          0.00
Status: Ótimo encontrado
Custo total (Reais): 481970.4143022338

P_T(1,1) = 200.0
P_T(1,2) = 200.0
P_T(1,3) = 200.0
P_T(1,4) = 200.0
P_T(1,5) = 200.0
P_T(1,6) = 200.0
P_T(1,7) = 200.0
P_T(1,8) = 200.0
P_T(1,9) = 200.0
P_T(1,10) = 200.0
P_T(1,11) = 200.0
P_T(1,12) = 200.0
P_T(2,1) = 50.0
P_T(2,2) = 60.0
P_T(2,3) = 40.0
P_T(2,4) = 55.0
P_T(2,5) = 70.0
P_T(2,6) = 63.485368295152846
P_T(2,7) = 49.85358189450391
P_T(2,8) = 59.848599898639605
P_T(2,9) = 44.87210574555126
P_T(2,10) = 54.860611579405216
P_T(2,11) = 59.843140116665516
P_T(2,12) = 69.85000526026948

P_H(1,1) = -0.0
P_H(1,2) = -0.0
P_H(1